<a href="https://colab.research.google.com/github/summermccune/Tokenization/blob/word2vec/Word2Vec%26SVM_bigrams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
from gensim.utils import simple_preprocess
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import glob
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import bigrams
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [48]:
def read_files(malwarefolder, malwareType):
  for sample in malwarefolder:
    with open(sample, 'r') as f:
      data = f.read().split()
      #make opcodes into bigrams
      bigrms = list(bigrams(data))
      #make the bigrams each their own string
      bigrms = [f"{first} {second}" for first, second in bigrms]
      data = bigrms
    #append sample to opcodes list in df
    df.loc[len(df)] = [data, malwareType]

In [49]:
#creating dataframe for samples and label
df = pd.DataFrame(columns = ['opcodes','label'])

#specify paths for each malware
winwebsec = glob.glob("/content/drive/MyDrive/Data/malware2/winwebsec/*.txt")
zbot = glob.glob("/content/drive/MyDrive/Data/malware2/zbot/*.txt")
zeroaccess = glob.glob("/content/drive/MyDrive/Data/malware2/zeroaccess/*.txt")

#read files
read_files(winwebsec, 0)
read_files(zbot, 1)
read_files(zeroaccess, 2)

print(df.shape)
df.head()


(7801, 2)


opcodes  label
0  [push mov, mov sub, sub push, push push, push ...      0
1  [push mov, mov sub, sub mov, mov mov, mov call...      0
2  [push mov, mov sub, sub xor, xor mov, mov push...      0
3  [push mov, mov sub, sub push, push push, push ...      0
4  [mov push, push mov, mov sub, sub lea, lea add...      0

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['label']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'label'}, axis=1)
              .sort_values('label', ascending=True))
  xs = counted['label']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_0.sort_values('label', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('label')
_ = plt.ylabel('count()')

In [50]:
#word2vec
model = Word2Vec(df['opcodes'], min_count=1, vector_size=100)

In [51]:
#SVM
X = np.array([np.mean([model.wv[word] for word in text if word in model.wv]
                        , axis=0) for text in df['opcodes']])
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
svm = SVC(kernel='rbf')
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)

#accuracy and conf matrix
print("Accuracy:", np.mean(y_pred == y_test))
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.9795134443021767
[[439   3   0]
 [  6 188   6]
 [  1   0 138]]
